### Importing the libraries

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

### Passing Open AI API Key

In [2]:
import getpass

api_key = getpass.getpass('OpenAI API Key:')

OpenAI API Key: ········


In [3]:
def split_doc(file):

    loader = TextLoader(file,encoding='utf8')
    docs = loader.load()

    from langchain.text_splitter import RecursiveCharacterTextSplitter

    chunk_size = 100

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=10)
    texts = text_splitter.split_documents(docs)

    n = len(texts)
    while not (n>=3 and n<6):
        chunk_size += 100
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=10)
        texts = text_splitter.split_documents(docs)
        n = len(texts)

    k= 1
    for i in range(0,n,2):
            for j in range(i,i+2):
                with open(f'./Data/Files/Krishna{k}.txt', 'a') as fp:
                    if j<n:
                        for item in (texts[j].page_content).split('\n'):
                            fp.write("%s\n" % item)
            k += 1

### Split and load the document and then split it into chunks

In [4]:
split_doc(r'./Data/Krishna.txt')

loader = DirectoryLoader('./Data/Files',glob='**/*.txt')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts = text_splitter.split_documents(docs)

### Selecting the type of embedding to use & creating the VectoreStore

In [5]:
embeddings = OpenAIEmbeddings(openai_api_key=api_key,model="text-embedding-ada-002")
db = Chroma.from_documents(texts, embeddings)

retriever = db.as_retriever()

### Creating a chain to answer questions

In [7]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever,return_source_documents=True)

### Querying

In [8]:
question = input('Enter your question: ')
output = qa(question)
output['result']

Enter your question:  Who is Lord Krishna?


' Lord Krishna is the 8th incarnation of Vishnu and the God of peace and love. Many people worship Krishna, including my family. He is known for fighting off demons in his childhood, killing Kamsa, a wicked ruler, accompanying the Pandavas in their exile, and giving lessons to Arjuna, a Panadava before a war.'